In [ ]:
import json
import pandas as pd

In [ ]:
# Load the dataset
with open('./data/unprocessed/pricesList.json') as pricef:
    priceData = json.load(pricef)

with open("./data/unprocessed/productList.json") as productf:
    products = json.load(productf)

with open("./data/unprocessed/dollarRate.csv") as dollarf:
    dollarRate = pd.read_csv(dollarf)

with open("./data/unprocessed/ceypetcoPrices.json") as ceypetcof:
    ceypetcoPrice = json.load(ceypetcof)

# Price Data Pre Process

In [ ]:
month_map = {
    '' : 0,
    'Jan': 1,
    'Feb': 2,
    'Mar': 3,
    'Apr': 4,
    'May': 5,
    'Jun': 6,
    'June': 6,
    'Jul': 7,
    'July': 7,
    'Aug': 8,
    'Sep': 9,
    'Oct': 10,
    'Nov': 11,
    'Dec': 12
}

In [ ]:
# initialize lists to store extracted products data
codeName = []
product = []
category = []

for p in products:
      codeName.append(p["product"])
      product.append(p["name"])
      category.append(p["category"])
    

In [ ]:
# Initialize lists to store extracted price data
names = []
prices = []
weeks = []
months = []
years = []
seasons = []

In [ ]:
# Iterate over each week's data
for week_data in priceData:
    # Extract the week, month, and year from the date
    # Ordinal Encoding
    date_parts = week_data['Date'].split('.')
    week = int(date_parts[0][1:])
    month = month_map.get(date_parts[1])
    year = int(date_parts[2])

    if month in range(3, 5):
        season = "First Inter-monsoon Season"
    elif month in range(5, 10):
        season = "South-West Monsoon Season"
    elif month in range(10, 12):
        season = "Second Inter-monsoon Season"
    else:
        season = "North-East Monsoon Season"
    
    # Iterate over each item in the week's data
    for item_name, price in week_data.items():
        # Skip the 'Date' key
        if item_name == 'Date':            
            continue
        # skip imported items
        if "Imported" in item_name:
            continue
        # skip SPICES and FISH items
        if "SPICES" in item_name:
            continue
        if "FISH" in item_name:
            continue
        if "Fish" in item_name:
            continue
        if "MEAT" in item_name:
            continue
        if "BAKERY" in item_name:
            continue
        if "MILK" in item_name:
            continue
        if "COCONUT" in item_name:
            continue
        if "EGGS" in item_name:
            continue
        if "FLOUR" in item_name:
            continue
        if ("Carrot_UPCVEG_1kg" not in item_name) and ("Leeks_UPCVEG_1kg" not in item_name) and ("Green_Chillies_LCVEG_1kg" not in item_name) and ("BeetRoot_UPCVEG_1kg" not in item_name):
            continue
        # Append extracted data to lists
        names.append(item_name)
        prices.append(price)
        weeks.append(week)
        months.append(month)
        years.append(year)
        seasons.append(season)

In [ ]:
# Create a DataFrame from the extracted data
df = pd.DataFrame({
    'Name': names,
    'Price': prices,
    'Week': weeks,
    'Month': months,
    'Year': years,
    "Season": seasons
})

df1 = pd.DataFrame({
    "code": codeName,
    "name": product,
    "category": category
})

In [ ]:
# Save the DataFrame to a CSV file
df.to_csv('./data/processed/pricesList.csv', index=False)
df1.to_csv('./data/processed/productList.csv', index=False)
print("CSV file saved successfully.")

# Fuel Price Pre Process

In [ ]:
# Initialize lists to store extracted price data
date = []
names = []
prices = []

In [ ]:
for prices_data in ceypetcoPrice:
    for item_name, price in prices_data.items():
        # Skip the 'Date' key
        if item_name == 'Date':            
            continue
        # Append extracted data to lists
        names.append(item_name)
        prices.append(price)
        date.append(prices_data["Date"])


In [ ]:
print(date.__len__())
print(names.__len__())
print(prices.__len__())

In [ ]:
df = pd.DataFrame({
    'Date': date,
    'Name': names,
    'Price': prices
})
df.set_index('Date', inplace=True)
df.reset_index(inplace=True)

In [ ]:
df.to_csv('./data/processed/ceypetcoPrices.csv', index=False)

# Rainfall Data Pre Process

In [ ]:
# Read the Excel file
data = pd.read_excel('./data/unprocessed/rainfallData.xls')

In [ ]:
data.columns = map(str.lower, data.columns)
# drop if id column is nan
data = data.dropna(subset=['id'])
# drop unwanted columns
data = data.drop(columns=['id', 'longitude', 'latitude', 'elevation', 'code', 'abbreviation'])
data.info()

In [ ]:
def month_to_column(month):
    months = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
    return months[month - 1]

In [ ]:
# Create an empty DataFrame to store the reshaped data
reshaped_data = pd.DataFrame(columns=['anuradhapura', 'jaffna', 'nuwaraeliya', 'year', 'month'])

# Iterate over years from 2017 to 2022
for year in range(2017, 2023):
    rows = data[data['yyyy'] == year].iterrows()
    # temp variable to store the all the stations month data
    anuradhapura = []
    jaffna = []
    nuwaraeliya = []
    # iterate over the rows and append month data
    for index, row in rows:
        for month in range(1, 13):
            if row['station_name'] == 'ANURADHAPURA':
                anuradhapura.append(row[month_to_column(month)])
            elif row['station_name'] == 'JAFFNA':
                jaffna.append(row[month_to_column(month)])
            elif row['station_name'] == 'NUWARA ELIYA':
                nuwaraeliya.append(row[month_to_column(month)])
    # iterate over the months and append the data to the reshaped DataFrame
    for month in range(1, 13):
        reshaped_data = pd.concat([reshaped_data, pd.DataFrame({'anuradhapura': [anuradhapura[month-1]], 'jaffna': [jaffna[month-1]], 'nuwaraeliya': [nuwaraeliya[month-1]], 'year': [year], 'month': [month]})])

In [ ]:
reshaped_data.to_csv('./data/processed/rainfallData.csv', index=False)